In [43]:
import pandas as pd
import keras
from keras.models import Sequential
from keras.layers import Dense
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix , accuracy_score
mlflow.set_tracking_uri('http://localhost:5000/')
ann = mlflow.create_experiment('ANN-bank')

In [2]:
%store -r df

In [3]:
%store -r X

In [4]:
y = df[['Exited']]

In [5]:
X.shape , y.shape

((10000, 13), (10000, 1))

In [6]:
def split_data(X ,y):    
    X_train ,  x_test,y_train ,y_test = train_test_split(X,y, test_size=0.2, random_state=0)
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    x_test = sc.fit_transform(x_test)
    return X_train , x_test , y_train , y_test

In [16]:
X_train , x_test , y_train , y_test = split_data(X, y)

In [17]:
X_train.shape , x_test.shape , y_train.shape , y_test.shape

((8000, 13), (2000, 13), (8000, 1), (2000, 1))

In [44]:
with mlflow.start_run(experiment_id=ann):

    classifier = Sequential()
    classifier.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu' , input_dim=13))
    classifier.add(Dense(output_dim=6 , kernel_initializer='uniform',activation='relu'))
    classifier.add(Dense(output_dim=1 , kernel_initializer='uniform' , activation='sigmoid'))
    classifier.compile(optimizer='adam', loss='binary_crossentropy' , metrics=['accuracy'])
    
    classifier.fit(X_train,y_train, batch_size=10 , epochs=100, verbose=2)
    y_pred = classifier.predict(x_test)
    y_pred = y_pred > 0.5
    cm = confusion_matrix(y_test,y_pred)
    accuracy = accuracy_score(y_test , y_pred) * 100
    mlflow.log_metric("accuracy", accuracy)
    print('Confusion Matrix:',cm)
    
    mlflow.sklearn.log_model(classifier, "ANN-Bank")

/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", input_dim=13, units=6)`
  after removing the cwd from sys.path.
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="relu", units=6)`
  """
/opt/anaconda3/envs/Churn/lib/python3.6/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(kernel_initializer="uniform", activation="sigmoid", units=1)`
  


Epoch 1/100
 - 1s - loss: 0.4850 - accuracy: 0.7956
Epoch 2/100
 - 1s - loss: 0.4148 - accuracy: 0.8216
Epoch 3/100
 - 1s - loss: 0.3986 - accuracy: 0.8286
Epoch 4/100
 - 1s - loss: 0.3877 - accuracy: 0.8305
Epoch 5/100
 - 1s - loss: 0.3780 - accuracy: 0.8401
Epoch 6/100
 - 1s - loss: 0.3711 - accuracy: 0.8461
Epoch 7/100
 - 1s - loss: 0.3658 - accuracy: 0.8497
Epoch 8/100
 - 1s - loss: 0.3621 - accuracy: 0.8518
Epoch 9/100
 - 1s - loss: 0.3591 - accuracy: 0.8536
Epoch 10/100
 - 1s - loss: 0.3564 - accuracy: 0.8536
Epoch 11/100
 - 1s - loss: 0.3546 - accuracy: 0.8545
Epoch 12/100
 - 1s - loss: 0.3529 - accuracy: 0.8565
Epoch 13/100
 - 1s - loss: 0.3525 - accuracy: 0.8558
Epoch 14/100
 - 1s - loss: 0.3508 - accuracy: 0.8545
Epoch 15/100
 - 1s - loss: 0.3499 - accuracy: 0.8556
Epoch 16/100
 - 1s - loss: 0.3484 - accuracy: 0.8564
Epoch 17/100
 - 1s - loss: 0.3484 - accuracy: 0.8544
Epoch 18/100
 - 1s - loss: 0.3470 - accuracy: 0.8569
Epoch 19/100
 - 1s - loss: 0.3467 - accuracy: 0.8594
Ep